In [276]:
import os
import pandas as pd
import json


pd.set_option('display.float_format', '{:.2f}'.format)


In [252]:


# Define the folder number
folder_number = '13'  # Change this to the specific folder number you are working with

# Define the paths to the JSON files
base_path = f'results_Aug1/slow_liquidity/'
channels_json_path = os.path.join(base_path, 'channels.json')
forwarding_history_json_path = os.path.join(base_path, 'forwarding_history.json')




In [253]:


# Load channels.json
with open(channels_json_path, 'r') as file:
    channels_data = json.load(file)

# Load forwarding_history.json
with open(forwarding_history_json_path, 'r') as file:
    forwarding_history_data = json.load(file)

# Extract the forwarding events
forwards = forwarding_history_data.get('forwards', [])

# Create the initial DataFrame
rows = []
for entry in forwards:
    if isinstance(entry, dict):
        rows.append({
            'addTimeNs': int(entry['addTimeNs']),
            'resolveTimeNs': int(entry.get('resolveTimeNs', 0)),
            'eventTimeNs': int(entry['addTimeNs']),
            'eventType': 'add',
            'incomingAmount': int(entry['incomingAmount']),
            'outgoingAmount': int(entry['outgoingAmount']),
            'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
            'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
        })
        if 'resolveTimeNs' in entry:
            rows.append({
                'addTimeNs': int(entry['addTimeNs']),
                'resolveTimeNs': int(entry['resolveTimeNs']),
                'eventTimeNs': int(entry['resolveTimeNs']),
                'eventType': 'resolve',
                'incomingAmount': int(entry['incomingAmount']),
                'outgoingAmount': int(entry['outgoingAmount']),
                'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
            })

pair_schedule_df = pd.DataFrame(rows)

# Sort the DataFrame by eventTimeNs
pair_schedule_df = pair_schedule_df.sort_values(by='eventTimeNs').reset_index(drop=True)

In [254]:
# Create a DataFrame for channel capacities
channels_df = pd.DataFrame(channels_data['channels'])
channels_df['chan_id'] = channels_df['chan_id'].astype(int)
channels_df['capacity'] = channels_df['capacity'].astype(int)*1000
channels_df = channels_df[['chan_id', 'capacity']]

In [337]:
def track_funds(pair_schedule_df, channels_df, jam_lim = 0.2):
    # Initialize a dictionary to track available funds
    #funds = {channel: {'incoming': cap // 2, 'outgoing': cap // 2} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}
    funds = {channel: {'incoming': 0, 'outgoing': 0} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}

    
    # List to store times when locked funds exceed 80%
    high_locked_times = []
    status = []
    for _, row in pair_schedule_df.iterrows():
        #if (row['shortChannelId_incoming'] not in {545357767442432, 551954837209088}) and (row['shortChannelId_outgoing'] not in {545357767442432, 551954837209088}):
        if True:
            if row['eventType'] == 'add':
                funds[row['shortChannelId_incoming']]['outgoing'] += row['incomingAmount']
                funds[row['shortChannelId_outgoing']]['incoming'] += row['outgoingAmount']
            elif row['eventType'] == 'resolve':
                funds[row['shortChannelId_incoming']]['outgoing'] -= row['incomingAmount']
                funds[row['shortChannelId_outgoing']]['incoming'] -= row['outgoingAmount']
 


            # Check if the locked amounts exceed 80% of available funds
            for channel, cap in zip(channels_df['chan_id'], channels_df['capacity']):
                status.append({
                        'time': row['eventTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'incoming_locked': funds[channel]['incoming'],
                        'outgoing_locked': funds[channel]['outgoing']
                    })
                if funds[channel]['incoming'] > jam_lim * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'resolve_time': row['resolveTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'direction': 'incoming',
                        'funds_locked': funds[channel]['incoming'],
                    })
                if funds[channel]['outgoing'] > jam_lim * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'resolve_time': row['resolveTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'direction': 'outgoing',
                        'funds_locked': funds[channel]['outgoing']   
                    })

    return pd.DataFrame(high_locked_times), pd.DataFrame(status)

liq_jam_ratio = 0.95
high_locked_df, st_df = track_funds(pair_schedule_df, channels_df, liq_jam_ratio)


In [338]:
pair_schedule_df.head(10)

,addTimeNs,resolveTimeNs,eventTimeNs,eventType,incomingAmount,outgoingAmount,shortChannelId_outgoing,shortChannelId_incoming
0,1721917088242157352,1721917093448108352,1721917088242157352,add,1053,53,393625162809344,378232000020480
1,1721917088242157352,1721917093448108352,1721917093448108352,resolve,1053,53,393625162809344,378232000020480
2,1721917112943667352,1721917115291763352,1721917112943667352,add,1053,53,393625162809344,378232000020480
3,1721917112943667352,1721917115291763352,1721917115291763352,resolve,1053,53,393625162809344,378232000020480
4,1721917142887081352,1721917145786102352,1721917142887081352,add,1053,53,393625162809344,378232000020480
5,1721917142887081352,1721917145786102352,1721917145786102352,resolve,1053,53,393625162809344,378232000020480
6,1721917166900195352,1721917174994031352,1721917166900195352,add,1053,53,393625162809344,378232000020480
7,1721917168061480352,1721917174996158352,1721917168061480352,add,1002,2,393625162809344,378232000020480
8,1721917166900195352,1721917174994031352,1721917174994031352,resolve,1053,53,393625162809344,378232000020480
9,1721917168061480352,1721917174996158352,1721917174996158352,resolve,1002,2,393625162809344,378232000020480


In [339]:
st_df.head()

,time,channel,capacity,incoming_locked,outgoing_locked
0,1721917088242157352,379331511648256,2000000000,0,0
1,1721917088242157352,346346162814976,1620000000,0,0
2,1721917088242157352,393625162809344,100000000000,53,0
3,1721917088242157352,404620279087104,25000000000,0,0
4,1721917088242157352,361739325603840,3000000000,0,0


In [340]:
high_locked_df.head(20)

,time,resolve_time,channel,capacity,direction,funds_locked
0,1722525625548198519,1722525625679134791,393625162809344,100000000000,incoming,50097102000
1,1722525625577741891,1722525625694320521,393625162809344,100000000000,incoming,50097102000
2,1722525625589251555,1722525625707815169,393625162809344,100000000000,incoming,55097108000
3,1722525625598018334,1722525625598018334,393625162809344,100000000000,incoming,50097102000
4,1722525656680169899,1722525656825483533,393625162809344,100000000000,incoming,50097102000
5,1722525656741308161,1722525656853009070,393625162809344,100000000000,incoming,50097102000
6,1722525688293844718,1722525688457920841,393625162809344,100000000000,incoming,50097102000
7,1722525688308363718,1722525688471880245,393625162809344,100000000000,incoming,55097108000
8,1722525688322716076,1722525688483128172,393625162809344,100000000000,incoming,60097114000
9,1722525688367946744,1722525688367946744,393625162809344,100000000000,incoming,55097108000


In [341]:
high_locked_df[high_locked_df['channel']==379331511648256]

,time,resolve_time,channel,capacity,direction,funds_locked


# count jammed time

In [342]:
def calculate_active_time(df):
    # Sort by start time
    df = df.sort_values(by='time')
    
    # Initialize variables
    total_active_time = 0
    current_start = None
    current_end = None
    
    for _, row in df.iterrows():
        start, end = row['time'], row['resolve_time']
        
        if current_start is None:
            current_start = start
            current_end = end
        else:
            if start <= current_end:
                # Overlapping period, extend the end time if needed
                current_end = max(current_end, end)
            else:
                # Non-overlapping period, add the previous period's duration
                total_active_time += current_end - current_start
                current_start = start
                current_end = end
    
    # Add the last period
    if current_start is not None:
        total_active_time += current_end - current_start
    
    return total_active_time

grouped = high_locked_df.groupby(['channel', 'direction'])

results = grouped.apply(calculate_active_time).reset_index()
results.columns = ['channel', 'direction', 'total_active_time']

# Step 5: Print the results
print(results)

           channel direction  total_active_time
0  393625162809344  incoming         2136654393


In [343]:
results['total_active_time_minutes'] = results['total_active_time']/60000000000

In [344]:
results

,channel,direction,total_active_time,total_active_time_minutes
0,393625162809344,incoming,2136654393,0.04
